In [1]:
'''
Using the netout function and reshape the netout into fpga format and back.
'''

__author__ = "Abarajithan G"
__copyright__ = "Copyright 2019, Final Year Project"
__credits__ = ["Abarajithan G"]
__version__ = "1.0.0"
__maintainer__ = "Abarajthan G"
__email__ = "abarajithan07@gmail.com"
__status__ = "Research"

In [2]:
import numpy as np
import pickle
import sys
sys.path.append("../../aba_framework/")
from utils import *

ANCHORS = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
NUM_CLASSES = 80
OBJ_THRESHOLD = 0.3
NMS_THRESHOLD = 0.3

In [3]:
boxes_real = pickle.load(open('data/boxes_list.pickle', 'rb'))

yolo_py_out = np.load('data/yolo_out_array.npy')
_, H, W, _ = yolo_py_out.shape
yolo_py_out.shape

(1, 8, 12, 425)

In [4]:
netout_py_input = np.reshape(yolo_py_out, (1, H, W, 5, 85))[0]
netout_py_input.shape

(8, 12, 5, 85)

In [5]:
boxes = decode_netout(netout = netout_py_input, 
              anchors = ANCHORS, 
              nb_class=NUM_CLASSES,
              obj_threshold=OBJ_THRESHOLD,
              nms_threshold=NMS_THRESHOLD)

../../aba_framework\utils.py:219: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


In [13]:
yolo_py_out = np.load('data/yolo_out_array.npy')
_, H, W, _ = yolo_py_out.shape
netout_py_input = np.reshape(yolo_py_out, (1, H, W, 5, 85))[0]

out_real = decode_netout(netout = netout_py_input, anchors = ANCHORS, nb_class=NUM_CLASSES,
                         obj_threshold=OBJ_THRESHOLD, nms_threshold=NMS_THRESHOLD)

In [14]:
yolo_py_out = np.load('data/yolo_out_array.npy')

f16_mem_tensor = np.einsum('ijkl->klj',yolo_py_out) # (1,8,12,425) -> (12,425,8)
f16_mem_array = f16_mem_tensor.flatten()
f16_mem_array_bytes = f16_mem_array.tobytes()
f = open('data/f16_yolo_out_mem_bytes.bin', 'wb')
f.write(f16_mem_array)


print('mem_tensor.shape = ',f16_mem_tensor.shape)
print('mem_array.shape = ', f16_mem_array.shape)
print('mem_array in Bytes: ', len(f16_mem_array_bytes))

f32_mem_array = f16_mem_array.astype(np.float32)
f32_mem_array_bytes = f32_mem_array.tobytes()
f = open('data/f32_yolo_out_mem_bytes.bin', 'wb')
f.write(f32_mem_array)

mem_tensor.shape =  (12, 425, 8)
mem_array.shape =  (40800,)
mem_array in Bytes:  81600


163200

In [15]:
f16_mem_tensor.shape

(12, 425, 8)

In [16]:
a = np.log10(np.abs(f16_mem_tensor))
a = a[~np.isnan(a)]
a = a[~np.isinf(a)]

min(a)

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


-5.418

In [14]:
f16_tensor_reshaped = f16_mem_tensor.reshape((12,5,85,8))
f16_tensor_reshaped.shape

(12, 5, 85, 8)

In [15]:
f16_tensor_reshaped[0,1,2,3]

-0.8057

In [16]:
f16_mem_tensor.shape

(12, 425, 8)

In [17]:
f16_mem_tensor.reshape((12,5,85,8))[0,0,3,4]

0.9873

In [18]:
f16_mem_array.shape
i16_mem_array = np.frombuffer(f16_mem_array.tobytes(), dtype=np.uint16)

In [19]:
with open('data/mem_array_f16.txt', 'w') as f:
    for i in range(f16_mem_array.size):
        f.write(str(f16_mem_array[i])+'\n')

In [20]:
with open('data/mem_array_i16.txt', 'w') as f:
    for i in range(i16_mem_array.size):
        f.write(str(i16_mem_array[i])+'\n')

In [21]:
netout_array = netout_py_input.flatten()
with open('data/netout_array_f16.txt', 'w') as f:
    for i in range(netout_array.size):
        f.write(str(netout_array[i])+'\n')

In [22]:
netout_array_i16 = np.frombuffer(netout_array.tobytes(), dtype=np.uint16)
with open('data/netout_array_i16.txt', 'w') as f:
    for i in range(netout_array_i16.size):
        f.write(str(netout_array_i16[i])+'\n')

In [26]:
f32_mem_array = f16_mem_array.astype(np.float32)
i32_mem_array = np.frombuffer(f32_mem_array.tobytes(), dtype=np.uint32)
f32_netout_array = netout_array.astype(np.float32)
i32_netout_array = np.frombuffer(f32_netout_array.tobytes(), dtype=np.uint32)


with open('data/mem_array_f32.txt', 'w') as f:
    for i in range(f32_mem_array.size):
        f.write(str(f32_mem_array[i])+'\n')
with open('data/mem_array_i32.txt', 'w') as f:
    for i in range(i32_mem_array.size):
        f.write(str(i32_mem_array[i])+'\n')
with open('data/netout_array_f32.txt', 'w') as f:
    for i in range(f32_netout_array.size):
        f.write(str(f32_netout_array[i])+'\n')
with open('data/netout_array_i32.txt', 'w') as f:
    for i in range(f32_netout_array.size):
        f.write(str(f32_netout_array[i])+'\n')

In [27]:
i16_all = np.arange(2**16, dtype=np.uint16)
f16_all = np.frombuffer(i16_all.tobytes(), dtype=np.float16)

f32_all = f16_all.astype(np.float32)
i32_all = np.frombuffer(f32_all.tobytes(), dtype=np.uint32)

with open('data/all_f32.txt', 'w') as f:
    for i in range(f32_all.size):
        f.write(str(f32_all[i])+'\n')
with open('data/all_i32.txt', 'w') as f:
    for i in range(i32_all.size):
        f.write(str(i32_all[i])+'\n')
with open('data/all_f32.bin', 'wb') as f:
    f.write(f32_all.tobytes())